# Analytics Mindset EDGAR Explorer 

This notebook contains the solution to Part 2 of the EDGAR explorer cases using Python. This solution will provide an example using Tesla, which has the CIK # 1318605. 

Note that CIK numbers are 10-digits long. If your company's CIK is less than 10-digits, then add leading zeros to get to 10-digits. In the example below, Tesla's CIK is entered as 0001318605, which is 10-digits long. 

In [59]:
import json
import requests
import pandas as pd


cik = "0001318605"
url = "https://data.sec.gov/submissions/CIK" + cik + ".json"
headers = {'User-Agent' : 'your university  youremail@university.edu'}
response = requests.get(url, headers=headers)
data=response.json()



Confirm that the JSON object returned is a dictionary:

In [ ]:
print(type(data))

Identify the keys associated with the top-level dictionary:

In [ ]:
data.keys()

Confirm the company's CIK and Name (note that the CIK will drop the leading zeros):

In [ ]:
data['cik']

In [ ]:
data['name']

and the data type for each of the values associated with these keys:

In [ ]:
for k,v in data.items():
    dt=type(data.items())
    print (k,dt)

Identify where the data relating to the type of forms filed by the entity are in the data object (hint: it is a list nested within several dictionaries). Note that in the prior answer, the final dictionary is called "filings <class 'dict_items'>" this is likely where the forms data is located as this will give information about the filings whereas the prior dictionaries tend to relate to entity-level facts like the company's cik, name and ticker symbol for example.

Start by saving the filings dictionary into a new data frame and rerunning the loop to identify dictionary keys:

In [ ]:
data2 = data['filings']
for k,v in data2.items():
    dt=type(data2.items())
    print (k,dt)

This is an example of a nested JSON data structure as referenced in the case. Note that python syntax allows a dictionary within a dictionary to be accessed by adding that disctionary to the end of the previous dictionary. For example, suppose we wanted to access the data in dictionary2 which is nested within dictionary1, we can use the following syntax: ['dictionary1']['dictionary2']. Next take a look at the data within the dictionary: files <class 'dict_items'>

In [ ]:
type(data['filings']['files'])

The data is stored as a list. This means that there is not another nested dictionary in this part of the file. Printing the file to the log will allow us to see what the list contains.

In [ ]:
data['filings']['files']

The print out highlights that this information is technical information regarding the number of filings summarized in this JSON document and any other associated files for the entity's filings (if they have more than 2000 filings they will have multiple entries in the list). This can be confirmed with the field 'filingCount'). This suggests that the forms data is within the field: recent <class 'dict_items'>

Repeating above:

In [ ]:
type(data['filings']['recent'])

The above code highlights that the field recent (nested within filings) is a dictionary. Saving the dictionary 'recent' into a new dataframe called data2 and then running the loop to understand which fields are nested within the recent dictionary can be done by repeating the code above:   

In [ ]:
data2 = data['filings']['recent']
for k,v in data2.items():
    dt=type(data2.items())
    print (k,dt)

We have now found the field 'form' which contains all of the forms filed by the company. The structure of the dictionary 'recent' is likely a dictionary of lists, as each of the fields above, such as accessionNumber, filingDate, form etc, are expected to contain information about multiple filings made by the entity. As we are interested in the field form, we will examine the data type of form, rembering that it is nested within recent which is nested within filings, similar to earlier we use the python syntax:

In [ ]:
type(data['filings']['recent']['form'])

Save the list of forms filed as a pandas dataframe (we will call it data3):

In [37]:
data3=pd.DataFrame(data['filings']['recent']['form'])

## Identify the most recently filed form and the dates and timestamps associated with that form.



Identify the number of unique forms filed by the entity. Note that the API returns forms in chronological order, but we can check this by looking at the field 'filingDate' which contains the date for each filing in the 'recent' dictionary, note that 'filingDate' is a list. To get the first item from a list, you can use the python syntax [0] and then each subsequent filing date will be ordered sequentially, i.e., the second filing is [1] etc. To get a range of filing dates from the list, we can use the python syntax [0:4] to recover the first five filing dates (the syntax can be modified to any range). Using that logic, to recover the first ten filing dates:    



In [ ]:
data['filings']['recent']['filingDate'][0:9]

In most cases, ten filings should highlight that the filings are ordered in chronological order, with the most recent filing first. To answer our question, we can identify the most recently filed form and the dates and timestamps associated with that form, as the data in space [0] for all of these fields. All of these fields are stored in the dataframe data2. So we can use the syntax data2['fieldName'][0] to recover the requested information:    

In [ ]:
data2['form'][0]

In [ ]:
data2['filingDate'][0]

In [ ]:
data2['acceptanceDateTime'][0]

## Using the list of forms filed by the entity (which you have saved as a pandas dataframe), use Python syntax to:
►	Identify the number of unique forms filed by the entity.
►	Calculate the number of times each form was filed based in this dataset and identify the most popular filing for your company.

Recall that we saved the field 'form' as a dataframe named data3 above. We will reference data3 to answer the two questions in this section. 

In [ ]:
uniqueFilings=pd.unique(data3)
print(uniqueFilings)

We could count the output above to see how many unique filings are in our file. It is also possible to use the python syntax len() to count the number for us. This will remove any possible human errors in counting and would be especially helpful if there are hundreds of unique filings.

In [ ]:
nUnique = len(uniqueFilings)
print(nUnique)

We can now go ahead and count the filings using the groupby and count commands. In most cases the most common filing is the form 4 filing. What is it for your company?

In [ ]:
data3.groupby(data3[0])[0].count()